### Load libraries and data

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [3]:
df1 = pd.read_csv('./preprocessed_log_data.csv')

In [14]:
df1['student_affordable'].value_counts()

student_affordable
False    1596
True      225
Name: count, dtype: int64

In [4]:
df1['energy_mark'].value_counts()

energy_mark
none    836
A15     243
A20     231
C       208
D       113
A10      82
B        68
E        29
F         8
G         3
Name: count, dtype: int64

In [9]:
836/len(df1)

0.45908841295991215

In [12]:
df1['energy_mark'].apply(lambda x: x[0]).map({'A':'A','B':'B','C':'C','D':'D','E':'E-G','F':'E-G','G':'E-G','n':'None'}).value_counts()

energy_mark
None    836
A       556
C       208
D       113
B        68
E-G      40
Name: count, dtype: int64

In [3]:
# Dropping the columns that are used to create total_monthly_rent_log
df.drop(columns=['monthly_rent_log', 'monthly_aconto_log'], inplace=True)

In [4]:
# Splitting the data into features and target for regression
X_reg_no_dummies = df.drop(columns=['total_monthly_rent_log']).copy()
y_reg = df['total_monthly_rent_log'].copy()

# Splitting the data into features and target for classification
X_cls_no_dummies = df.drop(columns=['months_on_website']).copy()
y_cls = df['months_on_website'].copy()

In [5]:
# Make the categorical variables into dummies
X_reg = pd.get_dummies(X_reg_no_dummies).to_numpy()
X_cls = pd.get_dummies(X_cls_no_dummies).to_numpy()

y_reg = y_reg.to_numpy()
y_cls = y_cls.to_numpy()

### Model development

In [60]:
import warnings
warnings.filterwarnings('ignore')

In [61]:
# Placeholder data - load your actual data here
X = X_reg  # Feature matrix
y = y_reg  # Target variable

# Define hyperparameter grids
lambda_values = [0.01, 0.1, 10, 100, 1000, 10000]  # Example values for regularization in Ridge
hidden_units_values = [1, 2, 4, 8, 16, 32]  # Example values for ANN hidden units

# Outer cross-validation
outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)
outer_results = []

# Outer CV loop
for outer_fold, (train_outer_idx, test_outer_idx) in enumerate(tqdm(outer_cv.split(X), desc="Outer CV")):
    X_train_outer, X_test_outer = X[train_outer_idx], X[test_outer_idx]
    y_train_outer, y_test_outer = y[train_outer_idx], y[test_outer_idx]

    # Standardize features based on outer train set
    scaler = StandardScaler()
    X_train_outer = scaler.fit_transform(X_train_outer)
    X_test_outer = scaler.transform(X_test_outer)

    # Inner cross-validation for hyperparameter tuning
    inner_cv = KFold(n_splits=10, shuffle=True, random_state=42)
    
    # Initialize placeholders for best models and errors
    best_ann_mse, best_linreg_mse = float('inf'), float('inf')
    best_h, best_lambda = None, None

    # ANN tuning
    for h in tqdm(hidden_units_values, desc="ANN Tuning"):
        ann_mses = []
        for train_inner_idx, val_inner_idx in inner_cv.split(X_train_outer):
            X_train_inner, X_val_inner = X_train_outer[train_inner_idx], X_train_outer[val_inner_idx]
            y_train_inner, y_val_inner = y[train_inner_idx], y[val_inner_idx]

            # Define ANN model
            model = lambda: nn.Sequential(
                nn.Linear(X_train_inner.shape[1], 2*h),
                nn.ReLU(),
                nn.Linear(2*h, h),
                nn.ReLU(),
                nn.Linear(h, 1),
            )
            ann_model = model()
            criterion = nn.MSELoss()
            optimizer = optim.Adam(ann_model.parameters(), lr=0.001)

            # Train the model
            ann_model.train()
            for epoch in range(1000):
                optimizer.zero_grad()
                outputs = ann_model(torch.tensor(X_train_inner, dtype=torch.float32))
                loss = criterion(outputs, torch.tensor(y_train_inner, dtype=torch.float32).view(-1, 1))
                loss.backward()
                optimizer.step()

            # Validate the model
            ann_model.eval()
            with torch.no_grad():
                y_pred_val = ann_model(torch.tensor(X_val_inner, dtype=torch.float32)).numpy()
            ann_mse = mean_squared_error(y_val_inner, y_pred_val)
            ann_mses.append(ann_mse)

        avg_ann_mse = np.mean(ann_mses)
        if avg_ann_mse < best_ann_mse:
            best_ann_mse = avg_ann_mse
            best_h = h

    # Linear regression tuning
    for lam in tqdm(lambda_values, desc="Linear Regression Tuning"):
        linreg_mses = []
        for train_inner_idx, val_inner_idx in inner_cv.split(X_train_outer):
            X_train_inner, X_val_inner = X_train_outer[train_inner_idx], X_train_outer[val_inner_idx]
            y_train_inner, y_val_inner = y[train_inner_idx], y[val_inner_idx]

            # Define linear model with regularization
            linreg_model = Ridge(alpha=lam)
            linreg_model.fit(X_train_inner, y_train_inner)
            y_pred_val = linreg_model.predict(X_val_inner)
            linreg_mse = mean_squared_error(y_val_inner, y_pred_val)
            linreg_mses.append(linreg_mse)

        avg_linreg_mse = np.mean(linreg_mses)
        if avg_linreg_mse < best_linreg_mse:
            best_linreg_mse = avg_linreg_mse
            best_lambda = lam

    # Train best models from inner loop on the entire outer training set
    ann_model = model()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(ann_model.parameters(), lr=0.01)
    ann_model.train()
    for epoch in range(1000):
        optimizer.zero_grad()
        outputs = ann_model(torch.tensor(X_train_outer, dtype=torch.float32))
        loss = criterion(outputs, torch.tensor(y_train_outer, dtype=torch.float32).view(-1, 1))
        loss.backward()
        optimizer.step()

    ann_model.eval()
    with torch.no_grad():
        y_pred_test_ann = ann_model(torch.tensor(X_test_outer, dtype=torch.float32)).numpy()
    test_mse_ann = mean_squared_error(np.exp(y_test_outer)+1, np.exp(y_pred_test_ann)+1)

    linreg_model = Ridge(alpha=best_lambda)
    linreg_model.fit(X_train_outer, y_train_outer)
    y_pred_test_linreg = linreg_model.predict(X_test_outer)
    test_mse_linreg = mean_squared_error(np.exp(y_test_outer)+1, np.exp(y_pred_test_linreg)+1)

    # Baseline model (predicting the mean)
    baseline_model = DummyRegressor(strategy="mean")
    baseline_model.fit(X_train_outer, y_train_outer)
    y_pred_test_baseline = baseline_model.predict(X_test_outer)
    test_mse_baseline = mean_squared_error(np.exp(y_test_outer)+1, np.exp(y_pred_test_baseline)+1)

    # Append results for this outer fold
    outer_results.append({
        'outer_fold': outer_fold + 1,
        'best_h': best_h,
        'test_mse_ann': test_mse_ann,
        'best_lambda': best_lambda,
        'test_mse_linreg': test_mse_linreg,
        'test_mse_baseline': test_mse_baseline
    })

# Create a DataFrame to display results in table format
results_df = pd.DataFrame(outer_results)
print(results_df)

Linear Regression Tuning: 100%|██████████| 6/6 [00:00<00:00, 32.87it/s]
Outer CV: 10it [15:05, 90.54s/it]

   outer_fold  best_h  test_mse_ann  best_lambda  test_mse_linreg  \
0           1      32  7.387754e+08        10000     1.964315e+10   
1           2      32  1.426222e+10        10000     1.670919e+10   
2           3      32  7.357393e+07        10000     2.399173e+10   
3           4      32  2.219569e+10        10000     4.111237e+10   
4           5      32  3.287433e+07        10000     3.036055e+07   
5           6      32  5.312566e+07        10000     2.399069e+10   
6           7      32  4.070376e+07        10000     1.609616e+07   
7           8      32  3.969305e+07        10000     4.425355e+07   
8           9      32  1.456825e+09        10000     1.393658e+10   
9          10      32  3.406446e+09        10000     1.535987e+10   

   test_mse_baseline  
0       1.986468e+10  
1       1.680635e+10  
2       2.426877e+10  
3       4.139120e+10  
4       5.063747e+07  
5       2.426621e+10  
6       3.131637e+07  
7       7.241897e+07  
8       1.403018e+10  
9       1.

### New attempt also with a GLM

In [76]:
# Define polynomial degree and lambda values for GLM
poly_degrees = [1, 2, 3]  # 1 for linear interactions, 2 for quadratic interactions
glm_lambda_values = [0.1, 1, 10, 100, 1000, 10000]
outer_results = []
# Outer CV loop
for outer_fold, (train_outer_idx, test_outer_idx) in enumerate(tqdm(outer_cv.split(X), desc="Outer CV")):
    X_train_outer, X_test_outer = X[train_outer_idx], X[test_outer_idx]
    y_train_outer, y_test_outer = y[train_outer_idx], y[test_outer_idx]

    # Standardize features based on outer train set
    scaler = StandardScaler()
    X_train_outer = scaler.fit_transform(X_train_outer)
    X_test_outer = scaler.transform(X_test_outer)

    # Inner cross-validation for hyperparameter tuning
    inner_cv = KFold(n_splits=10, shuffle=True, random_state=42)
    
    # Initialize placeholders for best models and errors
    best_ann_mse, best_linreg_mse, best_glm_mse = float('inf'), float('inf'), float('inf')
    best_h, best_lambda, best_glm_params = None, None, None

    # ANN tuning
    for h in tqdm(hidden_units_values, desc="ANN Tuning"):
        ann_mses = []
        for train_inner_idx, val_inner_idx in inner_cv.split(X_train_outer):
            X_train_inner, X_val_inner = X_train_outer[train_inner_idx], X_train_outer[val_inner_idx]
            y_train_inner, y_val_inner = y[train_inner_idx], y[val_inner_idx]

            # Define ANN model
            model = lambda: nn.Sequential(
                nn.Linear(X_train_outer.shape[1], 2 * h),
                nn.ReLU(),
                nn.Linear(2 * h, h),
                nn.ReLU(),
                nn.Linear(h, 1),
            )
            ann_model = model()
            criterion = nn.MSELoss()
            optimizer = optim.Adam(ann_model.parameters(), lr=0.001)

            # Train the model
            ann_model.train()
            for epoch in range(1000):
                optimizer.zero_grad()
                outputs = ann_model(torch.tensor(X_train_inner, dtype=torch.float32))
                loss = criterion(outputs, torch.tensor(y_train_inner, dtype=torch.float32).view(-1, 1))
                loss.backward()
                optimizer.step()

            # Validate the model
            ann_model.eval()
            with torch.no_grad():
                y_pred_val = ann_model(torch.tensor(X_val_inner, dtype=torch.float32)).numpy()
            ann_mse = mean_squared_error(y_val_inner, y_pred_val)
            ann_mses.append(ann_mse)

        avg_ann_mse = np.mean(ann_mses)
        if avg_ann_mse < best_ann_mse:
            best_ann_mse = avg_ann_mse
            best_h = h

    # Linear regression tuning
    for lam in tqdm(lambda_values, desc="Linear Regression Tuning"):
        linreg_mses = []
        for train_inner_idx, val_inner_idx in inner_cv.split(X_train_outer):
            X_train_inner, X_val_inner = X_train_outer[train_inner_idx], X_train_outer[val_inner_idx]
            y_train_inner, y_val_inner = y[train_inner_idx], y[val_inner_idx]

            # Define linear model with regularization
            linreg_model = Ridge(alpha=lam)
            linreg_model.fit(X_train_inner, y_train_inner)
            y_pred_val = linreg_model.predict(X_val_inner)
            linreg_mse = mean_squared_error(y_val_inner, y_pred_val)
            linreg_mses.append(linreg_mse)

        avg_linreg_mse = np.mean(linreg_mses)
        if avg_linreg_mse < best_linreg_mse:
            best_linreg_mse = avg_linreg_mse
            best_lambda = lam


    # GLM tuning with cross-join effects
    for degree in tqdm(poly_degrees, desc="GLM Tuning"):
        for lam in glm_lambda_values:
            glm_mses = []
            for train_inner_idx, val_inner_idx in inner_cv.split(X_train_outer):
                X_train_inner, X_val_inner = X_train_outer[train_inner_idx], X_train_outer[val_inner_idx]
                y_train_inner, y_val_inner = y[train_inner_idx], y[val_inner_idx]

                # Define GLM model with cross-join interactions
                glm_model = make_pipeline(
                    PolynomialFeatures(degree=degree, interaction_only=True, include_bias=False),
                    Ridge(alpha=lam)
                )
                glm_model.fit(X_train_inner, y_train_inner)
                y_pred_val = glm_model.predict(X_val_inner)
                glm_mse = mean_squared_error(y_val_inner, y_pred_val)
                glm_mses.append(glm_mse)

            avg_glm_mse = np.mean(glm_mses)
            if avg_glm_mse < best_glm_mse:
                best_glm_mse = avg_glm_mse
                best_glm_params = {'degree': degree, 'lambda': lam}

    # Train best models from inner loop on the entire outer training set
    best_ann_model = lambda: nn.Sequential(
                nn.Linear(X_train_outer.shape[1], 2 * best_h),
                nn.ReLU(),
                nn.Linear(2 * best_h, best_h),
                nn.ReLU(),
                nn.Linear(best_h, 1),
            )
    ann_model = best_ann_model()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(ann_model.parameters(), lr=0.01)
    ann_model.train()
    for epoch in range(1000):
        optimizer.zero_grad()
        outputs = ann_model(torch.tensor(X_train_outer, dtype=torch.float32))
        loss = criterion(outputs, torch.tensor(y_train_outer, dtype=torch.float32).view(-1, 1))
        loss.backward()
        optimizer.step()

    ann_model.eval()
    with torch.no_grad():
        y_pred_test_ann = ann_model(torch.tensor(X_test_outer, dtype=torch.float32)).numpy()
    test_mse_ann = mean_squared_error(np.exp(y_test_outer)+1, np.exp(y_pred_test_ann)+1)

    #Linear model
    linreg_model = Ridge(alpha=best_lambda)
    linreg_model.fit(X_train_outer, y_train_outer)
    y_pred_test_linreg = linreg_model.predict(X_test_outer)
    test_mse_linreg = mean_squared_error(np.exp(y_test_outer)+1, np.exp(y_pred_test_linreg)+1)

    # Baseline model (predicting the mean)
    baseline_model = DummyRegressor(strategy="mean")
    baseline_model.fit(X_train_outer, y_train_outer)
    y_pred_test_baseline = baseline_model.predict(X_test_outer)
    test_mse_baseline = mean_squared_error(np.exp(y_test_outer)+1, np.exp(y_pred_test_baseline)+1)

    # Train best GLM model on the outer training set
    glm_model = make_pipeline(
        PolynomialFeatures(degree=best_glm_params['degree'], interaction_only=True, include_bias=False),
        Ridge(alpha=best_glm_params['lambda'])
    )
    glm_model.fit(X_train_outer, y_train_outer)
    y_pred_test_glm = glm_model.predict(X_test_outer)
    test_mse_glm = mean_squared_error(np.exp(y_test_outer)+1, np.exp(y_pred_test_glm)+1)

    # Append results for this outer fold
    outer_results.append({
        'outer_fold': outer_fold + 1,
        'best_h': best_h,
        'test_mse_ann': test_mse_ann,
        'best_lambda': best_lambda,
        'test_mse_linreg': test_mse_linreg,
        'test_mse_baseline': test_mse_baseline,
        'best_glm_params': best_glm_params,
        'test_mse_glm': test_mse_glm
    })
    print(outer_results)
# Create a DataFrame to display results in table format
results1_df = pd.DataFrame(outer_results)

GLM Tuning: 100%|██████████| 3/3 [03:17<00:00, 65.92s/it]
Outer CV: 1it [04:55, 295.29s/it]

[{'outer_fold': 1, 'best_h': 32, 'test_mse_ann': 738775384.165011, 'best_lambda': 10000, 'test_mse_linreg': 19643148183.13463, 'test_mse_baseline': 19864679985.78159}, {'outer_fold': 2, 'best_h': 32, 'test_mse_ann': 14262217148.607422, 'best_lambda': 10000, 'test_mse_linreg': 16709188939.313482, 'test_mse_baseline': 16806352404.63012}, {'outer_fold': 3, 'best_h': 32, 'test_mse_ann': 73573934.93377702, 'best_lambda': 10000, 'test_mse_linreg': 23991725642.129875, 'test_mse_baseline': 24268767874.281}, {'outer_fold': 4, 'best_h': 32, 'test_mse_ann': 22195685982.575184, 'best_lambda': 10000, 'test_mse_linreg': 41112373598.84125, 'test_mse_baseline': 41391200795.43109}, {'outer_fold': 5, 'best_h': 32, 'test_mse_ann': 32874327.04789609, 'best_lambda': 10000, 'test_mse_linreg': 30360551.52572315, 'test_mse_baseline': 50637469.84446768}, {'outer_fold': 6, 'best_h': 32, 'test_mse_ann': 53125658.79258171, 'best_lambda': 10000, 'test_mse_linreg': 23990686935.699535, 'test_mse_baseline': 242662144

GLM Tuning: 100%|██████████| 3/3 [03:32<00:00, 70.84s/it]
Outer CV: 2it [09:59, 300.28s/it]

[{'outer_fold': 1, 'best_h': 32, 'test_mse_ann': 738775384.165011, 'best_lambda': 10000, 'test_mse_linreg': 19643148183.13463, 'test_mse_baseline': 19864679985.78159}, {'outer_fold': 2, 'best_h': 32, 'test_mse_ann': 14262217148.607422, 'best_lambda': 10000, 'test_mse_linreg': 16709188939.313482, 'test_mse_baseline': 16806352404.63012}, {'outer_fold': 3, 'best_h': 32, 'test_mse_ann': 73573934.93377702, 'best_lambda': 10000, 'test_mse_linreg': 23991725642.129875, 'test_mse_baseline': 24268767874.281}, {'outer_fold': 4, 'best_h': 32, 'test_mse_ann': 22195685982.575184, 'best_lambda': 10000, 'test_mse_linreg': 41112373598.84125, 'test_mse_baseline': 41391200795.43109}, {'outer_fold': 5, 'best_h': 32, 'test_mse_ann': 32874327.04789609, 'best_lambda': 10000, 'test_mse_linreg': 30360551.52572315, 'test_mse_baseline': 50637469.84446768}, {'outer_fold': 6, 'best_h': 32, 'test_mse_ann': 53125658.79258171, 'best_lambda': 10000, 'test_mse_linreg': 23990686935.699535, 'test_mse_baseline': 242662144

GLM Tuning: 100%|██████████| 3/3 [03:20<00:00, 66.89s/it]
Outer CV: 3it [14:52, 297.30s/it]

...

[{'outer_fold': 1, 'best_h': 32, 'test_mse_ann': 738775384.165011, 'best_lambda': 10000, 'test_mse_linreg': 19643148183.13463, 'test_mse_baseline': 19864679985.78159}, {'outer_fold': 2, 'best_h': 32, 'test_mse_ann': 14262217148.607422, 'best_lambda': 10000, 'test_mse_linreg': 16709188939.313482, 'test_mse_baseline': 16806352404.63012}, {'outer_fold': 3, 'best_h': 32, 'test_mse_ann': 73573934.93377702, 'best_lambda': 10000, 'test_mse_linreg': 23991725642.129875, 'test_mse_baseline': 24268767874.281}, {'outer_fold': 4, 'best_h': 32, 'test_mse_ann': 22195685982.575184, 'best_lambda': 10000, 'test_mse_linreg': 41112373598.84125, 'test_mse_baseline': 41391200795.43109}, {'outer_fold': 5, 'best_h': 32, 'test_mse_ann': 32874327.04789609, 'best_lambda': 10000, 'test_mse_linreg': 30360551.52572315, 'test_mse_baseline': 50637469.84446768}, {'outer_fold': 6, 'best_h': 32, 'test_mse_ann': 53125658.79258171, 'best_lambda': 10000, 'test_mse_linreg': 23990686935.699535, 'test_mse_baseline': 242662144

GLM Tuning: 100%|██████████| 3/3 [03:26<00:00, 68.88s/it]
Outer CV: 9it [56:08, 373.78s/it]

[{'outer_fold': 1, 'best_h': 32, 'test_mse_ann': 738775384.165011, 'best_lambda': 10000, 'test_mse_linreg': 19643148183.13463, 'test_mse_baseline': 19864679985.78159}, {'outer_fold': 2, 'best_h': 32, 'test_mse_ann': 14262217148.607422, 'best_lambda': 10000, 'test_mse_linreg': 16709188939.313482, 'test_mse_baseline': 16806352404.63012}, {'outer_fold': 3, 'best_h': 32, 'test_mse_ann': 73573934.93377702, 'best_lambda': 10000, 'test_mse_linreg': 23991725642.129875, 'test_mse_baseline': 24268767874.281}, {'outer_fold': 4, 'best_h': 32, 'test_mse_ann': 22195685982.575184, 'best_lambda': 10000, 'test_mse_linreg': 41112373598.84125, 'test_mse_baseline': 41391200795.43109}, {'outer_fold': 5, 'best_h': 32, 'test_mse_ann': 32874327.04789609, 'best_lambda': 10000, 'test_mse_linreg': 30360551.52572315, 'test_mse_baseline': 50637469.84446768}, {'outer_fold': 6, 'best_h': 32, 'test_mse_ann': 53125658.79258171, 'best_lambda': 10000, 'test_mse_linreg': 23990686935.699535, 'test_mse_baseline': 242662144

GLM Tuning: 100%|██████████| 3/3 [03:29<00:00, 69.68s/it]
Outer CV: 10it [1:01:10, 367.04s/it]

[{'outer_fold': 1, 'best_h': 32, 'test_mse_ann': 738775384.165011, 'best_lambda': 10000, 'test_mse_linreg': 19643148183.13463, 'test_mse_baseline': 19864679985.78159}, {'outer_fold': 2, 'best_h': 32, 'test_mse_ann': 14262217148.607422, 'best_lambda': 10000, 'test_mse_linreg': 16709188939.313482, 'test_mse_baseline': 16806352404.63012}, {'outer_fold': 3, 'best_h': 32, 'test_mse_ann': 73573934.93377702, 'best_lambda': 10000, 'test_mse_linreg': 23991725642.129875, 'test_mse_baseline': 24268767874.281}, {'outer_fold': 4, 'best_h': 32, 'test_mse_ann': 22195685982.575184, 'best_lambda': 10000, 'test_mse_linreg': 41112373598.84125, 'test_mse_baseline': 41391200795.43109}, {'outer_fold': 5, 'best_h': 32, 'test_mse_ann': 32874327.04789609, 'best_lambda': 10000, 'test_mse_linreg': 30360551.52572315, 'test_mse_baseline': 50637469.84446768}, {'outer_fold': 6, 'best_h': 32, 'test_mse_ann': 53125658.79258171, 'best_lambda': 10000, 'test_mse_linreg': 23990686935.699535, 'test_mse_baseline': 242662144

In [77]:
len(outer_results)

30

### Format results 

In [40]:
results_df.columns = ['Outer Fold', 'Best Hidden Units', 'Test MSE ANN', 'Best Lambda', 'Test MSE LinReg', 'Test MSE Baseline']

In [55]:
results_df['Test MSE ANN'] = results_df['Test MSE ANN'].apply(lambda x: round(x/10**8, 2))

results_df['Test MSE LinReg'] = results_df['Test MSE LinReg'].apply(lambda x: round(x/10**8, 2))

results_df['Test MSE Baseline'] = results_df['Test MSE Baseline'].apply(lambda x: round(x/10**8, 2))

In [59]:
results_df.to_latex()

'\\begin{tabular}{lllllll}\n\\toprule\n & Outer Fold & Best Hidden Units & Test MSE ANN & Best Lambda & Test MSE LinReg & Test MSE Baseline \\\\\n\\midrule\n0 & 1 & 32 & 2.3e+08 & 10 & 181.56e+08 & 198.65e+08 \\\\\n1 & 2 & 32 & 142.94e+08 & 10 & 160.59e+08 & 168.06e+08 \\\\\n2 & 3 & 32 & 1.37e+08 & 10 & 220.74e+08 & 242.69e+08 \\\\\n3 & 4 & 32 & 195.35e+08 & 10 & 395.08e+08 & 413.91e+08 \\\\\n4 & 5 & 32 & 0.33e+08 & 10 & 0.1e+08 & 0.51e+08 \\\\\n5 & 6 & 32 & 2.08e+08 & 10 & 220.11e+08 & 242.66e+08 \\\\\n6 & 7 & 32 & 0.72e+08 & 10 & 0.07e+08 & 0.31e+08 \\\\\n7 & 8 & 32 & 0.37e+08 & 10 & 0.18e+08 & 0.72e+08 \\\\\n8 & 9 & 32 & 4.74e+08 & 10 & 131.32e+08 & 140.3e+08 \\\\\n9 & 10 & 32 & 33.32e+08 & 10 & 142.3e+08 & 155.31e+08 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [ ]:
results1_df.iloc[20:,:]

### Check best model

In [21]:
# Define ANN model
model = lambda: nn.Sequential(
    nn.Linear(X_train_inner.shape[1], 2*h),
    nn.ReLU(),
    nn.Linear(2*h, h),
    nn.ReLU(),
    nn.Linear(h, 1),
)

In [22]:
X_train_outer, X_test_outer = X[train_outer_idx], X[test_outer_idx]
y_train_outer, y_test_outer = y[train_outer_idx], y[test_outer_idx]

# Standardize features based on outer train set
scaler = StandardScaler()
X_train_outer = scaler.fit_transform(X_train_outer)
X_test_outer = scaler.transform(X_test_outer)


# Train best models from inner loop on the entire outer training set
ann_model = model()
criterion = nn.MSELoss()
optimizer = optim.Adam(ann_model.parameters(), lr=0.01)
ann_model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = ann_model(torch.tensor(X_train_outer, dtype=torch.float32))
    loss = criterion(outputs, torch.tensor(y_train_outer, dtype=torch.float32).view(-1, 1))
    loss.backward()
    optimizer.step()

ann_model.eval()
with torch.no_grad():
    y_pred_test_ann = ann_model(torch.tensor(X_test_outer, dtype=torch.float32)).numpy()
test_mse_ann = mean_squared_error(np.exp(y_test_outer)+1, np.exp(y_pred_test_ann)+1)



In [31]:
pred_vs_true = pd.concat([pd.DataFrame(np.exp(y_test_outer)+1), pd.DataFrame(np.exp(y_pred_test_ann)+1)], axis=1)
pred_vs_true.columns = ['true', 'pred']

In [32]:
pred_vs_true['diff'] = abs(pred_vs_true['true'] - pred_vs_true['pred'])

In [34]:
pred_vs_true.describe()

,true,pred,diff
count,1.820000e+02,1.820000e+02,182.000000
mean,2.673741e+04,2.236883e+04,7494.889828
std,1.242160e+05,1.107363e+05,58272.845221
min,4.302000e+03,2.270754e+02,1.513672
25%,1.102700e+04,9.886339e+03,425.035645
50%,1.350200e+04,1.268412e+04,1769.262695
75%,1.675075e+04,1.727384e+04,4065.020752
max,1.497352e+06,1.503605e+06,786547.137695


In [19]:
results_df['test_mse_ann'][0]

230355871.84476316

### Test for best model with Setup II

In [64]:
# Define ANN model
def create_ann_model(input_size, h):
    return nn.Sequential(
        nn.Linear(input_size, 2 * h),
        nn.ReLU(),
        nn.Linear(2 * h, h),
        nn.ReLU(),
        nn.Linear(h, 1),
    )

# Function to evaluate models
def evaluate_models(X, y, n_splits=10, h=16, lambda_reg=10000):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    ann_errors = []
    ridge_errors = []
    baseline_errors = []
    
    # Baseline prediction (mean)
    baseline_prediction = np.mean(y)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Standardize features
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Train Ridge Regression
        ridge_model = Ridge(alpha=lambda_reg)
        ridge_model.fit(X_train, y_train)
        ridge_pred = ridge_model.predict(X_test)
        ridge_errors.append(mean_squared_error(y_test, ridge_pred))

        # Train ANN
        ann_model = create_ann_model(X_train.shape[1], h)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(ann_model.parameters(), lr=0.01)
        ann_model.train()

        # Training Loop
        for epoch in range(1000):
            optimizer.zero_grad()
            outputs = ann_model(torch.tensor(X_train, dtype=torch.float32))
            loss = criterion(outputs, torch.tensor(y_train, dtype=torch.float32).view(-1, 1))
            loss.backward()
            optimizer.step()

        # Evaluation
        ann_model.eval()
        with torch.no_grad():
            y_pred_ann = ann_model(torch.tensor(X_test, dtype=torch.float32)).numpy()
        ann_errors.append(mean_squared_error(y_test, y_pred_ann))

        # Calculate Baseline Error
        baseline_errors.append(mean_squared_error(y_test, [baseline_prediction]*len(y_test)))

    return np.array(ann_errors), np.array(ridge_errors), np.array(baseline_errors)


# Evaluate models
h_value = 16  # Hidden units for ANN
lambda_value = 10000  # Regularization parameter for Ridge
ann_errors, ridge_errors, baseline_errors = evaluate_models(X, y, h=h_value, lambda_reg=lambda_value)

# Function to perform correlated t-test
def correlated_t_test(model_a_errors, model_b_errors):
    differences = model_a_errors - model_b_errors
    mean_diff = np.mean(differences)
    std_diff = np.std(differences, ddof=1)
    J = len(differences)
    
    # t-statistic
    t_stat = mean_diff / (std_diff / np.sqrt(J))
    df = J - 1
    p_value = 2 * stats.t.cdf(-np.abs(t_stat), df)
    
    # Confidence interval
    alpha = 0.05
    ci_low = mean_diff - stats.t.ppf(1 - alpha / 2, df) * (std_diff / np.sqrt(J))
    ci_high = mean_diff + stats.t.ppf(1 - alpha / 2, df) * (std_diff / np.sqrt(J))
    
    return mean_diff, std_diff, p_value, (ci_low, ci_high)

# Pairwise comparisons
results = {}

# ANN vs Ridge Regression
results['ANN vs Ridge Regression'] = correlated_t_test(ann_errors, ridge_errors)

# ANN vs Baseline
results['ANN vs Baseline'] = correlated_t_test(ann_errors, baseline_errors)

# Ridge Regression vs Baseline
results['Ridge Regression vs Baseline'] = correlated_t_test(ridge_errors, baseline_errors)

# Print results
for comparison, (mean_diff, std_diff, p_value, ci) in results.items():
    print(f"{comparison}:")
    print(f"  Mean Difference: {mean_diff:.4f}, Std. Dev: {std_diff:.4f}, P-Value: {p_value:.4f}")
    print(f"  Confidence Interval: {ci}")
    print()

ANN vs Ridge Regression:
  Mean Difference: 0.1403, Std. Dev: 0.1760, P-Value: 0.0328
  Confidence Interval: (0.01435395317052729, 0.2662226767224303)

ANN vs Baseline:
  Mean Difference: -0.0011, Std. Dev: 0.2020, P-Value: 0.9871
  Confidence Interval: (-0.14556631252528773, 0.14344198020223697)

Ridge Regression vs Baseline:
  Mean Difference: -0.1414, Std. Dev: 0.0354, P-Value: 0.0000
  Confidence Interval: (-0.16667743729056955, -0.11602352492543878)

